In [1]:
# import sys
# import os
# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

In [2]:
import numpy as np
from my_stuff import MufexKeys
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.enums import *
from quantfreedom.nb_funcs.nb_strategies.nb_strategy import *
from quantfreedom.nb_funcs.nb_simulate_base import nb_sim_backtest
from quantfreedom.helper_funcs import dl_ex_candles, candles_to_df

mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

In [3]:
candles = mufex_main.get_candles(
    symbol="BTCUSDT",
    timeframe="5m",
    candles_to_dl=10500,
)

In [4]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)

backtest_settings = BacktestSettings(
    array_size=1000,
    gains_pct_filter=0,
    total_trade_filter=40,
    upside_filter=0,
)

# backtest_settings = BacktestSettings()
dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([2, 4, 6, 8]) * 3,
    max_trades=np.array([5]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2, 4, 8, 10]),
    sl_based_on_add_pct=np.array([0.05, 0.1, 0.2]),
    sl_based_on_lookback=np.array([20, 40, 60]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.05, 0.1, 0.5, 1, 2]),
    trail_sl_when_pct=np.array([0.5, 1, 3]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=100,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="limit",
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)

In [5]:
bt_strat_df = nb_sim_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=mufex_main.exchange_settings,
    logger_bool=False,
    long_short="long",
    nb_strat_evaluate=nb_strat_long_evaluate,
    nb_strat_get_current_ind_settings=nb_strat_get_current_ind_settings,
    nb_strat_get_ind_set_str=nb_strat_get_ind_set_str,
    nb_strat_get_total_ind_settings=nb_strat_get_total_ind_settings,
    nb_strat_ind_creator=nb_strat_bt_create_ind,
    static_os=static_os,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 6
Total order settings to test: 2,160
Total combinations of settings to test: 12,960
Total candles: 10,500
Total candles to test: 136,080,000


In [7]:
bt_strat_df.sort_values(by=["qf_score"], ascending=False)

,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,qf_score,fees_paid,total_pnl,ending_eq
8,5,1883,130.0,21,109,19.808,16.154,0.551,9268.829,198.081,1198.081
7,5,1877,130.0,21,109,4.100,16.154,0.501,10610.293,41.002,1041.002
0,2,1343,100.0,17,83,23.608,17.000,0.404,5888.235,236.078,1236.078
3,4,803,130.0,23,107,87.961,17.692,0.385,9276.782,879.612,1879.612
1,4,257,90.0,19,71,30.475,21.111,0.259,2814.727,304.749,1304.749
4,5,257,90.0,19,71,30.475,21.111,0.259,2814.727,304.749,1304.749
5,5,263,88.0,19,69,78.336,21.591,0.081,3263.380,783.359,1783.359
2,4,263,96.0,20,76,62.893,20.833,0.029,3521.820,628.934,1628.934
6,5,803,112.0,22,90,309.874,19.643,0.014,10205.220,3098.740,4098.740
